In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
sys.path.append('../..')

In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import copy
import yaml
import datetime
from typing import Tuple, Dict

from simulator_external.simulator.simulator import *
from simulator_external.simulator.load_data import load_md_from_file
from stoikov import StoikovStrategy
from simulator_external.simulator.get_info import get_metrics, md_to_dataframe

In [13]:
MARKET_DATA_PATH = '../../data/1/btcusdt:Binance:LinearPerpetual/'

# Stoikov strategy

Article: [Stoikov 2008] Avellaneda, M., & Stoikov, S. (2008). High-frequency trading in a limit order book. Quantitative Finance, 8(3), 217-224.

**1. Reservation price**
$$ r(s, t) = s - q \gamma \sigma^2 (T-t). $$

- $s$ is mid-price.
- $q$ is position size in base asset.
- $T$ is terminal time.
- $t$ is current time.
- $\sigma$ is volatiliaty of standard Brownian motion ($dS = \sigma dW$), by which the price is modeled in this article.
- $\gamma$ -- risk aversity.

$\sigma$ and $\gamma$ are parameters of the strategy.

**2. Spread**

$$ \delta^a + \delta^b = \gamma\sigma^2(T-t) + \frac{2}{\gamma} \log\left(1 + \frac{\gamma}{k}\right).$$

Here $k = \alpha K$, where
- $\alpha$ is from $f^Q(x) \propto x^{-1-\alpha}$ -- density of market order size (formula 9 from [Stoikov 2008]);
- $K$ is from $\Delta p \propto K^{-1} \log(Q)$ (formulas 11, 12 from [Stoikov 2008]).

# Mid-price from market data

![BTC mid-price](../../eda/pics/btc/midprice.png)

# [Optional] Calculate parameter $\sigma$

In [14]:
# midprices = get_midprices(md)
# increments = midprices[1:] - midprices[:-1]
# sigma = np.std(increments)
# print(f'σ = {sigma}')

# Visualize results of the simulation

In [15]:
def plot_metrics(metrics, metrics_numeric, title=''):
    fig = make_subplots(rows=5, cols=1, row_heights=[0.3, 0.3, 0.2, 0.2, 0.2], shared_xaxes=True, vertical_spacing=0.005)
    fig.add_trace(go.Scatter(
        name='PnL',
        x=metrics['receive_ts'],
        y=metrics['worth_quote'],
        line=dict(color=px.colors.qualitative.Plotly[0]),
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        name='Mid-price',
        x=metrics['receive_ts'],
        y=metrics['mid_price'],
        line=dict(color=px.colors.qualitative.Plotly[5]),
        showlegend=False
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        name='Volume',
        x=metrics['receive_ts'],
        y=metrics['volume'],
        line=dict(color=px.colors.qualitative.Plotly[2]),
        showlegend=False
    ), row=3, col=1)

    fig.add_trace(go.Scatter(
        name='Base balance',
        x=metrics['receive_ts'],
        y=metrics['base_balance'],
        line=dict(color=px.colors.qualitative.Plotly[3]),
        showlegend=False
    ), row=4, col=1)

    fig.add_trace(go.Scatter(
        name='Quote balance',
        x=metrics['receive_ts'],
        y=metrics['quote_balance'],
        line=dict(color=px.colors.qualitative.Plotly[4]),
        showlegend=False
    ), row=5, col=1)

    metrics_numeric_txt = yaml.dump(metrics_numeric, default_flow_style=False, sort_keys=False).replace('\n', '<br>')
    fig.add_annotation(text=metrics_numeric_txt,
                       xref='paper', yref='paper', x=1, y=0.9,
                       align='left', font={'family': 'monospace'})

    time_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    title += f'<br>Backtest launch time: {time_str}'

    fig.update_layout(width=1000, height=1400, hovermode='x unified',
                      title=title, xaxis5_title='Receive time',
                      yaxis1_title='PnL (account worth in quote asset)', yaxis2_title='Mid-price',
                      yaxis3_title='Trade volume', yaxis4_title='Base asset balance',
                      yaxis5_title='Quote asset balance')
    fig.update_traces(xaxis='x1')
    return fig

# Run the backtesting

In [16]:
def my_round(x, ndigits):
    return round(float(x), ndigits)

def backtest(md, sim_params: dict, strat_params: dict, fee: float) -> Tuple[pd.DataFrame, Dict]:
    """Run backtest and return results.
    Args:
        md: Parameter `md` (market data) for `Sim` constructor.
        sim_params: All parameters for `Sim` constructor, except `md`
        strat_params: All parameters for StoikovStrategy constructor, except `sim`.
        fee: Parameter `fee` for `get_metrics()` function.
    Returns:
        Simulation metrics in form of time series and single quantities.
    """
    sim = Sim(md, **sim_params)
    strategy = StoikovStrategy(sim, **strat_params)
    trades_list, md_list, updates_list, all_orders = strategy.run()
    metrics = get_metrics(updates_list, fee=fee)
    metrics_numeric = {
        'Final PnL':    my_round(metrics['worth_quote'].iloc[-1], 2),
        'Final volume': my_round(metrics['volume'].iloc[-1], 3),
        'Base balance': {
            'min':    my_round(metrics['base_balance'].min(), 3),
            'max':    my_round(metrics['base_balance'].max(), 3),
            'median': my_round(metrics['base_balance'].median(), 3),
        },
        'Quote balance': {
            'min':    my_round(metrics['quote_balance'].min(), 2),
            'max':    my_round(metrics['quote_balance'].max(), 2),
            'median': my_round(metrics['quote_balance'].median(), 2),
        }
    }
    return metrics, metrics_numeric

## Compare simulations with different values of $\gamma$

In [17]:
MIN_TS = pd.Timestamp('2022-06-23 15:35:00')
MAX_TS = pd.Timestamp('2022-06-23 19:00:00')

In [18]:
md = load_md_from_file(MARKET_DATA_PATH, MIN_TS, MAX_TS)

In [19]:
sim_params = {
    'execution_latency': 10_000_000,
    'md_latency': 10_000_000
}
strat_params = {
    'gamma': None,  # Will be set in the loop
    'k': 0.8,  # calculated from BTCUSDT market data (see eda/eda-btc.ipynb)
    'sigma': 1,
    'adjust_delay': 1_000_000,
    'order_size': 0.001,
    'min_order_size': 0.001,
    'precision': 2
}
FEE = 0.0002

In [ ]:
metrics_list = []
metrics_numeric_list = []
fig_list = []
GAMMA_VALUES = [0.002, 0.005, 0.04, 0.01, 0.1, 0.5, 1, 2, 4]

for gamma in tqdm(GAMMA_VALUES):
    strat_params['gamma'] = gamma
    metrics, metrics_numeric = backtest(md, sim_params, strat_params, fee=FEE)
    metrics_list.append(metrics)
    metrics_numeric_list.append(metrics_numeric)

    fig = plot_metrics(metrics[::60], metrics_numeric,
                       title=f'gamma: {strat_params["gamma"]}, k: {strat_params["k"]}, '
                             f'sigma: {strat_params["sigma"]}, fee: {FEE*100}%')
    img_name = f'gamma_{gamma}.png'
    fig.write_image(f'backtesting_results/{img_name}')

with open('backtesting_results/input_params.yaml', 'w') as f:
    strat_params_copy = copy.deepcopy(strat_params)
    strat_params_copy['gamma'] = GAMMA_VALUES
    params = {
        'Simulator parameters': sim_params,
        'Strategy parameters': strat_params_copy,
        'Maker/taker fee': FEE
    }
    f.write(yaml.dump(params, default_flow_style=False, sort_keys=False))

## Backtesting results

The results are saved in `backtesting_results`. Interactive plots can be drawn with `fig.show()`. Pre-computed results for this notebook can be found in `backtesting_resultsing/compare_gamma`.

![gamma_2.png](backtesting_results/compare_gamma/gamma_2.png)